In [208]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [209]:
from langchain.chat_models import ChatOpenAI
llm_name = 'gpt-3.5-turbo'
llm = ChatOpenAI(model_name = llm_name, temperature = 0)

In [210]:
llm.predict("Hello world!")

'Hello! How can I assist you today?'

In [211]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path = '../dat/526963_processed_data.csv')
sections = loader.load()

In [212]:
len(sections)

74

In [213]:
section = sections[22].page_content
print(section)

section_reference: 3.2.4
category: Property Characteristics
section: Site Improvements
subsection: Landscape and Irrigation
cost_related: False
docu_txt: Was There An Irrigation System? is An underground automatic irrigation system was provided., Is there landscaping or irrigation? is yes, Describe landscaping issue(s) and recommendation(s) in complete sentences and identify specific locations is Based on the CC&R's and discussions with the management company, all "common" landscape areas are expensed through the HOA fees. All landscape areas within the subject's general area is ownership responsibility. Based on this, barren turf was observed throughout the site typical of conditions noted to the south elevation of the structure. Refurbishment of the turf is recommended.
Trees, shrubs and Ivy (growing on the walls) were observed impeding on the building rooftop and façade at points along the north, south and east walls. No portion of any vegetation should be in contact with the buildi

In [214]:
index = section.find('docu_txt')
len(section[index :])

1393

In [215]:
sections[0].metadata

{'source': '../dat/526963_processed_data.csv', 'row': 0}

In [216]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate


In [217]:
file = '../dat/526963_processed_data.csv'

In [225]:
# load documents
loader = CSVLoader(file_path = file)
sections = loader.load()

# split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
secs = text_splitter.split_documents(sections)

# define embedding
embeddings = OpenAIEmbeddings()

# create vector database from data
vectordb = Chroma.from_documents(sections, embeddings)

# define retriever
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# prompt
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Provide a thorough explanation of your reasoning, using bullet points for clarity where needed.
Conclude each response with 'Thank you for asking, Is there anything else I can help you with?' placed 1 line after the answer.

{context}

Return the output in the following format: 
Question: {question}
Answer:

Thank you for asking, Is there anything else I can help you with?
"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

#ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    verbose = True,
    combine_docs_chain_kwargs={'prompt': QA_CHAIN_PROMPT}
)

In [226]:
question = "What is the client info of the report?"
result = qa({"question": question})
print(result['answer'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Provide a thorough explanation of your reasoning, using bullet points for clarity where needed.
Conclude each response with 'Thank you for asking, Is there anything else I can help you with?' placed 1 line after the answer.

section_reference: 1.3
category: Property Description
section: CLIENT INFO
subsection: 
cost_related: False
docu_txt: Client State is California, Client Company is GK Management Co., Inc, Client Zip is 90230, Does the client require a professional stamp? is no, Client First Name is Jason, Client Last Name is Mandler, Client Address is 5150 Overland Avenue, Client City is Culver City

section_reference: 1.3
category: Property Description
section: CLIENT INFO
subsection: 
cost_related: False
docu_txt:

In [227]:
question = "What is the client's company?"
result = qa({"question": question})
print(result['answer'])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the client info of the report?
Assistant: Question: What is the client info of the report?
Answer: 
- Client State: California
- Client Company: GK Management Co., Inc
- Client Zip: 90230
- Does the client require a professional stamp?: No
- Client First Name: Jason
- Client Last Name: Mandler
- Client Address: 5150 Overland Avenue
- Client City: Culver City

Thank you for asking, Is there anything else I can help you with?
Follow Up Input: What is the client's company?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try 

In [228]:
question = "Who is Jason Mandler?"
result = qa({"question": question})
print(result["answer"])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the client info of the report?
Assistant: Question: What is the client info of the report?
Answer: 
- Client State: California
- Client Company: GK Management Co., Inc
- Client Zip: 90230
- Does the client require a professional stamp?: No
- Client First Name: Jason
- Client Last Name: Mandler
- Client Address: 5150 Overland Avenue
- Client City: Culver City

Thank you for asking, Is there anything else I can help you with?
Human: What is the client's company?
Assistant: Question: What is the client's company?
Answer: GK Management Co., Inc

Thank you for asking, Is there anything else I can help you with?
Follow Up Input: Who is Jason Mandler?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChai

In [229]:
question = "What is the REVIEWER INFO of the report?"
result = qa({"question": question})
print(result["answer"])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the client info of the report?
Assistant: Question: What is the client info of the report?
Answer: 
- Client State: California
- Client Company: GK Management Co., Inc
- Client Zip: 90230
- Does the client require a professional stamp?: No
- Client First Name: Jason
- Client Last Name: Mandler
- Client Address: 5150 Overland Avenue
- Client City: Culver City

Thank you for asking, Is there anything else I can help you with?
Human: What is the client's company?
Assistant: Question: What is the client's company?
Answer: GK Management Co., Inc

Thank you for asking, Is there anything else I can help you with?
Human: Who is Jason Mandler?
Assistant: Question: Who is Jason Mandler?
Answer: Jason Mandler is the client whose information is provided in the docu

In [230]:
question = "What is the overview of the property?"
result = qa({"question": question})
print(result["answer"])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the client info of the report?
Assistant: Question: What is the client info of the report?
Answer: 
- Client State: California
- Client Company: GK Management Co., Inc
- Client Zip: 90230
- Does the client require a professional stamp?: No
- Client First Name: Jason
- Client Last Name: Mandler
- Client Address: 5150 Overland Avenue
- Client City: Culver City

Thank you for asking, Is there anything else I can help you with?
Human: What is the client's company?
Assistant: Question: What is the client's company?
Answer: GK Management Co., Inc

Thank you for asking, Is there anything else I can help you with?
Human: Who is Jason Mandler?
Assistant: Question: Who is Jason Mandler?
Answer: Jason Mandler is the client whose information is provided in the docu

In [231]:
question = "What is the total cost?"
result = qa({"question": question})
print(result['answer'])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the client info of the report?
Assistant: Question: What is the client info of the report?
Answer: 
- Client State: California
- Client Company: GK Management Co., Inc
- Client Zip: 90230
- Does the client require a professional stamp?: No
- Client First Name: Jason
- Client Last Name: Mandler
- Client Address: 5150 Overland Avenue
- Client City: Culver City

Thank you for asking, Is there anything else I can help you with?
Human: What is the client's company?
Assistant: Question: What is the client's company?
Answer: GK Management Co., Inc

Thank you for asking, Is there anything else I can help you with?
Human: Who is Jason Mandler?
Assistant: Question: Who is Jason Mandler?
Answer: Jason Mandler is the client whose information is provided in the docu